In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../../')

## Regress SPY returns on Fama French and Sanity Check

In [3]:
from util.ff_util import read_ff_factors_daily, get_ff_factors_with_freq
from util.file_util import (
    StockTwitsFileReader, StockDataFileReader
)

In [4]:
stock_data_reader = StockDataFileReader()

In [5]:
start_date = '2017-01-01'
end_date = '2019-08-10'

In [6]:
def get_weekly_returns_for_ticker(ticker,
                                  start_date,
                                  end_date):
    price_ts = stock_data_reader.read_stockdata_in_range(ticker,
                                                         start_date,
                                                         end_date)
    weekly_price_ts = price_ts.resample('W-FRI').last()
    weekly_return_ts = weekly_price_ts.pct_change()
    weekly_return_ts['ticker'] = ticker
    return weekly_return_ts

In [7]:
# price_ts = stock_data_reader.read_stockdata_in_range('SPY',
#                                                     start_date,
#                                                     end_date)

In [ ]:
return_ts = price_ts.pct_change()
return_ts.columns = ['spy']
return_ts = return_ts.iloc[1:]

In [8]:
return_ts = get_weekly_returns_for_ticker('SPY', start_date, end_date).dropna()

In [ ]:
ff_daily_df = read_ff_factors_daily()

In [ ]:
ff_daily_df /= 100

In [9]:
ff_df = get_ff_factors_with_freq()

In [ ]:
# ff_daily_df.columns = ['mkt_rf', 'SMB', 'HML', 'RF']

In [ ]:
# ff_daily_df['Mkt-RF'] /= 100

In [10]:
merged = return_ts.merge(ff_df,
                left_index=True,
                right_index=True)

In [12]:
import statsmodels.formula.api as smf

In [13]:
lm = smf.ols("Q('adjusted close') ~ Q('Mkt-RF') + SMB + HML", data=merged).fit()

In [14]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Q('adjusted close')   R-squared:                       0.997
Model:                             OLS   Adj. R-squared:                  0.997
Method:                  Least Squares   F-statistic:                 1.453e+04
Date:                 Mon, 04 Nov 2019   Prob (F-statistic):          4.70e-165
Time:                         03:59:57   Log-Likelihood:                 740.12
No. Observations:                  135   AIC:                            -1472.
Df Residuals:                      131   BIC:                            -1461.
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0002    9.1e-05      2.281      0.024    2.75e-05       0.000
Q('Mkt-RF')     0.9745      0.005    208.116      0.000       0.965       0.984
SMB            -0.1333      0.008    -15.894      0.000      -0.150      -0.117
HML            -0.0197      0.008     -2.489      0.014      -0.035      -0.004
==============================================================================
Omnibus:                        0.117   Durbin-Watson:                   2.417
Prob(Omnibus):                  0.943   Jarque-Bera (JB):                0.268
Skew:                          -0.039   Prob(JB):                        0.874
Kurtosis:                       2.796   Cond. No.                         98.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""